<a href="https://colab.research.google.com/github/AdamMohsen4/FreightFixer/blob/ml/freightFixer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Necessary libraries
import numpy as np
import padas as pd
from google.colab import files
import torch

In [ ]:
# Utilize gpu if available
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [ ]:

uploaded = files.upload()

# Assuming the uploaded file is named 'your_file.csv' or 'your_file.xlsx'
filename = list(uploaded.keys())[0]

# Load CSV file
if filename.endswith('.csv'):
    df = pd.read_csv(filename)
    print("Loaded CSV file successfully!")

# Load Excel file
elif filename.endswith('.xlsx') or filename.endswith('.xls'):
    df = pd.read_excel(filename)
    print("Loaded Excel file successfully!")

else:
    raise ValueError("Unsupported file type. Please upload a CSV or Excel file.")
